# Averages daily nldas soilMoisture
- from hourly nc files 

In [2]:
import dask
import xarray
import numpy 
import pandas as pd
import datetime

In [3]:
def average_hourly(current_dt, downloads_dir):
    '''
    takes base file path and averages from hourly to daily using xarray resampling
    inputs:
    datetime as current_dt 
    outputs:
    daily average in appropriate subdirectory
    '''
    

In [5]:
start_dt = datetime.datetime(1979, 6, 1, 0, 0)

In [7]:
start_dt.month

6